## INTRO TO LLMS 
### MODULE 1 HOMEWORK


In [38]:
# Import libraries

import requests
import json
from elasticsearch import Elasticsearch, exceptions
from tqdm.auto import tqdm
import tiktoken

In [2]:
# Read JSON file and extract document details

with open('./documents.json', 'rt') as f_in:
    documents_file = json.load(f_in)

documents = []

for course in documents_file:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

### Question 1

In [9]:
es = Elasticsearch("http://localhost:9200")
es.info()

ObjectApiResponse({'name': '7bcdfcbcb96e', 'cluster_name': 'docker-cluster', 'cluster_uuid': '8D6V0FBmSla1jbPMHCuDcg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

### Question 2

In [10]:
# Create an index

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [11]:
index_name = "course-questions"

try:
    if not es.indices.exists(index=index_name):
        es.indices.create(index=index_name, body=index_settings)
        print("Index created successfully")
    else:
        print("Index already exists")
except exceptions.ConnectionError as e:
    print(f"Failed to create index: {e}")

Index created successfully


In [12]:
# Index the documents

for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

100%|███████████████████████████████████████████████████████████████| 948/948 [00:26<00:00, 35.75it/s]


### Question 3 

In [18]:
def retrieve_documents(query, index_name="course-questions", max_results=5):
    es = Elasticsearch("http://localhost:9200")
    
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            
            }
        }
    }

    try:
        response = es.search(index=index_name, body=search_query)
        if 'hits' in response and 'hits' in response['hits'] and len(response['hits']['hits']) > 0:
            top_result = response['hits']['hits'][0]
            top_score = top_result['_score']
            print(f"Top ranking result score: {top_score}")
            documents = [hit['_source'] for hit in response['hits']['hits']]
            return documents
        else:
            print("No results found.")
            return []
    except Exception as e:
        print(f"Error during search: {e}")
        return []


In [19]:
query = "How do I execute a command in a running docker container?"
documents = retrieve_documents(query)

if documents:
    top_result = documents[0]
    print(f"Top ranking result: {top_result}")
else:
    print("No documents found.")

Top ranking result score: 84.050095
Top ranking result: {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}


### Question 4

In [24]:
# Define a function that retrieves documents and matches user queries

def retrieve_documents_2(query, index_name, max_results=3):
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es.search(index=index_name, body=search_query)
    documents = [hit['_source'] for hit in response['hits']['hits']]
    return documents

In [31]:
query = "How do I execute a command in a running docker container?"

response_2 = retrieve_documents_2(query, index_name)

for doc in response_2:
    print(f"Question: {doc['question']}\nAnswer: {doc['text']}\n\n")

Question: How do I debug a docker container?
Answer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)


Question: How do I copy files from my local machine to docker container?
Answer: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani


Question: How do I copy files from a different folder into docker container’s working directory?
Answer: You can copy files from your local machine into a Docker container using

### Question 5

In [32]:
# Define a function to format the context
def format_context(response_2):
    context_entries = []
    context_template = """
Q: {question}
A: {text}
""".strip()
    
    for resp in response_2:
        context_entry = context_template.format(question=resp['question'], text=resp['text'])
        context_entries.append(context_entry)
    
    context = "\n\n".join(context_entries)
    return context

In [33]:
context = format_context(response_2)

In [35]:
# Construct the final prompt using the provided prompt_template

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

# Fill in the template with the question and context
final_prompt = prompt_template.format(question=query, context=context)

In [36]:
prompt_length = len(final_prompt)
print(f"Length of the final prompt: {prompt_length} characters")

Length of the final prompt: 1462 characters


### Question 6

In [39]:
# Calculate and print the number of tokens

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(final_prompt)
num_tokens = len(tokens)
print(f"Number of tokens in the final prompt: {num_tokens}")

Number of tokens in the final prompt: 322
